<a href="https://colab.research.google.com/github/AyushmanRaha/BizLens-Analytics/blob/main/notebooks/BizLens_Analytics_NoteBook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Basic Setup and Exploration**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import io
import requests
import zipfile
import gzip

In [ ]:
def displaySettings():
  pd.set_option('display.max_columns', None)
  pd.set_option('display.width', 1000)

In [ ]:
displaySettings()

# **Data Ingestion Engine**

# **DataFrame Setup and Modifications Functions**

**1) load_data() - For loading the data**

In [ ]:
def load_data(file_path, sheet_name=None):

  def is_url(path):
    return path.startswith(("http://", "https://", "ftp://"))

  if is_url(file_path):
    response = requests.get(file_path)
    response.raise_for_status()
    content = io.BytesIO(response.content)
  else:
    content = file_path

  file_lower = file_path.lower()

  try:
    if file_lower.endswith('.zip'):
      with zipfile.ZipFile(content, 'r') as z:
        file_inside = z.namelist()[0]
        with z.open(file_inside) as f:
          if file_inside.endswith('.csv'):
            df = pd.read_csv(f)
          elif file_inside.endswith(('.xls', '.xlsx')):
            df = pd.read_excel(f, sheet_name=sheet_name)
          else:
            raise ValueError("Unsupported file inside ZIP.")

    elif file_lower.endswith('.gz'):
      with gzip.open(content, 'rt') as f:
        df = pd.read_csv(f)

    elif file_lower.endswith('.csv'):
        df = pd.read_csv(content)

    elif file_lower.endswith(('.xls', '.xlsx')):
        df = pd.read_excel(content, sheet_name=sheet_name)

    else:
        raise ValueError("Unsupported file type. Provide a CSV, Excel, ZIP, or GZ file.")

    print(f"Succesfully loaded data from: {file_path}")
    return df

  except Exception as e:
    print(f"Error loading data from {file_path}: {e}")
    return None

**2) initial_exploration() - For initial analysis**

In [ ]:
def initial_exploration(df: pd.DataFrame):
  separator = "="*80
  print("\n" + separator)
  print("🔍 DATAFRAME OVERVIEW")
  print(separator)
  print(f"Shape: {df.shape[0]} rows x {df.shape[1]} columns")
  print(separator)
  print("\n📋 INFO")
  print(separator)
  df.info()
  print("\n📊 SUMMARY STATISTICS")
  print(separator)
  print(df.describe(include='all').T)
  print("\n👀 SAMPLE ROWS (HEAD)")
  print(separator)
  print(df.head())
  print("\n✅ END OF EXPLORATION")
  print(separator)

**3) clean_and_correct_data() - Removing empty and non numeric values from the dataFrame**

In [ ]:
def clean_and_correct_data(df, dropped_columns=None, numeric_columns=None, fill_value=0, verbose=True):

  #Creating a copy of the original dataFrame
  df1 = df.copy()

  # Changing the column names to lower case
  df1.columns = [c.strip().lower() for c in df1.columns]

  if numeric_columns is not None:
    numeric_columns_list = [c.strip().lower() for c in numeric_columns] # Changing to lower case
  else:
    numeric_columns_list = df1.select_dtypes(include=['int64', 'float64']).columns.tolist()
    if verbose:
      print(f"Auto-detected possible numeric columns: {numeric_columns_list}")

  if dropped_columns is not None:
    dropped_columns_list = [c.strip().lower() for c in dropped_columns] # Changing to lower case

  if dropped_columns is not None:
    df1 = df1.drop(dropped_columns_list, axis=1)

  #Changing all the values to numeric type and handling empty values.
  for col in numeric_columns_list:
    df1[col] = pd.to_numeric(df1[col], errors='coerce')
    df1[col] = df1[col].fillna(fill_value)

  null_summary = df1.isnull().sum()
  has_null = df1.isnull().any().any()

  if verbose:
    print("\nData Cleaning Summary:")
    print("-" * 50)
    print("Dropped Columns:")
    a=0
    for col in dropped_columns:
      a=a+1
      print(f"{a}) {col}")
    print("-" * 50)
    print(f"Null values remaining in the DataFrame: {has_null}")
    print("-" * 50)
    print(f"Shape: {df1.shape[0]} rows x {df1.shape[1]} columns")
    if has_null:
      print("\nColumns with remaining nulls:\n", null_summary[null_summary > 0])
    print("-" * 50)
    print(f"Updated Column list:")
    c=0
    for col in df1.columns.tolist():
      c=c+1
      print(f"{c}) {col}, dtype: {df1[col].dtype}")
    print("-" * 50)

  return df1 # Returning the modified dataFrame

**4) checkDataFrameFormat(original_df) - Checks if the provided DataFrame contains all mandatory columns.**

In [ ]:
def checkDataFrameFormat(original_df):

    df_copy = original_df.copy()

    # Define the mandatory columns as a set
    mandatory_cols = {
        'seniority', 'seniorcitizen',
        'gender',
        'partner',
        'dependents',
        'tenure',
        'monthly charges', 'monthlycharges',
        'total charges', 'totalcharges',
        'churn'
    }

    # Get the columns from the dataframe
    df_cols = set(df_copy.columns)

    # Find which mandatory columns are missing from the dataframe
    # logic: (Required) - (what we have) = (what is missing)
    missing_cols = mandatory_cols - df_cols

    # If the missing_cols set is not empty, we have a problem
    if missing_cols:
        print("ALERT: The following mandatory columns are missing from the DataFrame:")
        for col in missing_cols:
            print(f" - {col}")
        return False

    # If we pass the check
    return True

# **Churn Prediction and Analysis Functions**

**1) analyze_churn_permutations() - Used for creating a new DataFrame which calculates churning counts for each categories.**

In [ ]:
def analyze_churn_permutations(file_path):

    # Define Mandatory Columns
    mandatory_cols = {
        'seniority', 'seniorcitizen',
        'gender',
        'partner',
        'dependents',
        'tenure',
        'monthly charges', 'monthlycharges',
        'total charges', 'totalcharges',
        'churn'
    }

    # Identify Service Columns
    service_cols = [col for col in df.columns if col not in mandatory_cols]

    # 3. Preprocess Demographics
    # Handle Seniority
    if 'seniority' in df.columns:
        df['age category'] = df['seniority'].apply(lambda x: 'Senior' if x == 1 else 'Non-senior')
    elif 'seniorcitizen' in df.columns:
        df['age category'] = df['seniorcitizen'].apply(lambda x: 'Senior' if x == 1 else 'Non-senior')
    else:
        df['age category'] = 'Unknown'

    # Handle Gender with 'Other'
    unique_genders = set(df['gender'].unique())
    unique_genders.add('Other')
    df['gender'] = pd.Categorical(df['gender'], categories=list(unique_genders))

    # 4. Determine Services Opted per Row
    def get_services_opted(row):
        opted = []
        for col in service_cols:
            val = str(row[col])
            val_lower = val.lower()
            if val_lower == 'yes':
                opted.append(col)
            elif val_lower.startswith('no'):
                continue
            else:
                opted.append(col)
        return ", ".join(sorted(opted)) if opted else "None"

    df['services opted'] = df.apply(get_services_opted, axis=1)

    # 5. Grouping and Aggregation
    group_cols = ['gender', 'age category', 'partner', 'dependents', 'services opted']
    df['count'] = 1

    # Group by demographics/services and Churn
    grouped = df.groupby(group_cols + ['churn'], observed=False)['count'].sum().unstack(fill_value=0)

    # Rename Churn columns
    rename_map = {}
    if 'Yes' in grouped.columns: rename_map['Yes'] = 'Churned'
    if 'No' in grouped.columns: rename_map['No'] = 'Not Churned'
    grouped.rename(columns=rename_map, inplace=True)

    if 'Churned' not in grouped.columns: grouped['Churned'] = 0
    if 'Not Churned' not in grouped.columns: grouped['Not Churned'] = 0

    # Reset index
    grouped = grouped.reset_index()

    # 6. Calculate Metrics
    grouped['Total'] = grouped['Churned'] + grouped['Not Churned']
    grouped = grouped[grouped['Total'] > 0].copy()
    grouped['Churn %'] = (grouped['Churned'] / grouped['Total'] * 100).round(2)

    # 7. Final Formatting
    final_cols = ['gender', 'age category', 'partner', 'dependents', 'services opted', 'Churn %', 'Churned', 'Not Churned']
    result_df = grouped[final_cols]

    # Add Serial No.
    result_df.insert(0, 'Serial No.', range(1, len(result_df) + 1))

    # REMOVE THE CHURN COLUMN INDEX NAME (The word 'churn' above the index)
    result_df.columns.name = None

    return result_df

**2) Veriication function: For analyze_churn_permutations() function**

In [ ]:
def verify_churn_logic(df_input):
    """
    Verifies the churn analysis logic by taking a raw DataFrame as input,
    re-calculating the summary, and cross-referencing random samples
    against the input DataFrame.
    """
    # Create a working copy to avoid modifying the original dataframe
    df = df_input.copy()

    # Define mandatory cols (to identify services)
    mandatory_cols = {
        'seniority', 'seniorcitizen',
        'gender',
        'partner',
        'dependents',
        'tenure',
        'monthly charges', 'monthlycharges',
        'total charges', 'totalcharges',
        'churn'
    }
    service_cols = [col for col in df.columns if col not in mandatory_cols]

    # 3. Process Demographics
    if 'seniority' in df.columns:
        df['age category'] = df['seniority'].apply(lambda x: 'Senior' if x == 1 else 'Non-senior')
    elif 'seniorcitizen' in df.columns:
        df['age category'] = df['seniorcitizen'].apply(lambda x: 'Senior' if x == 1 else 'Non-senior')
    else:
        df['age category'] = 'Unknown'

    # 4. Determine Services Opted per Row
    def get_services_opted(row):
        opted = []
        for col in service_cols:
            val = str(row[col])
            val_lower = val.lower()
            if val_lower == 'yes':
                opted.append(col)
            elif val_lower.startswith('no'):
                continue
            else:
                opted.append(col)
        return ", ".join(sorted(opted)) if opted else "None"

    df['services opted'] = df.apply(get_services_opted, axis=1)

    # 5. Grouping
    group_cols = ['gender', 'age category', 'partner', 'dependents', 'services opted']
    df['count'] = 1

    # Group and pivot
    grouped = df.groupby(group_cols + ['churn'], observed=False)['count'].sum().unstack(fill_value=0)

    # Rename Churn columns
    rename_map = {}
    if 'Yes' in grouped.columns: rename_map['Yes'] = 'Churned'
    if 'No' in grouped.columns: rename_map['No'] = 'Not Churned'
    grouped.rename(columns=rename_map, inplace=True)

    # Ensure columns exist
    if 'Churned' not in grouped.columns: grouped['Churned'] = 0
    if 'Not Churned' not in grouped.columns: grouped['Not Churned'] = 0

    # Reset index and Calculate Total
    grouped = grouped.reset_index()
    grouped['Total'] = grouped['Churned'] + grouped['Not Churned']
    grouped = grouped[grouped['Total'] > 0].copy()

    result_df = grouped.reset_index(drop=True)

    # --- Step 2: Verification ---
    print(f"Total permutations generated: {len(result_df)}")

    # Sample 5 random rows (or fewer if dataset is small)
    sample_size = min(5, len(result_df))
    sample_indices = np.random.choice(result_df.index, sample_size, replace=False)

    results = []

    for idx in sample_indices:
        row = result_df.iloc[idx]

        # Filter the working df (which has the calculated 'services opted' and normalized columns)
        # This checks if the aggregation logic matches the row-wise data
        mask = (
            (df['gender'] == row['gender']) &
            (df['age category'] == row['age category']) &
            (df['partner'] == row['partner']) &
            (df['dependents'] == row['dependents']) &
            (df['services opted'] == row['services opted'])
        )

        subset = df[mask]

        # Calculate actual counts from the subset
        actual_churned = subset[subset['churn'] == 'Yes'].shape[0]
        actual_not_churned = subset[subset['churn'] == 'No'].shape[0]

        match = (actual_churned == row['Churned']) and (actual_not_churned == row['Not Churned'])

        results.append({
            'Permutation Index': idx,
            'Profile': f"{row['gender']}, {row['age category']}, P:{row['partner']}, D:{row['dependents']}",
            'Services': row['services opted'][:30] + "..." if len(row['services opted']) > 30 else row['services opted'],
            'Code Churned': row['Churned'],
            'Actual Churned': actual_churned,
            'Code Not Churned': row['Not Churned'],
            'Actual Not Churned': actual_not_churned,
            'Match': match
        })

    return pd.DataFrame(results)

# **Balance Sheet Analysis Functions**

# **Income Statement Analysis Functions**

# **Cash Flow Statement Analysis Functions**

# **Initial exploration and Data Cleaning/Corrections**

**Loading the data**

In [ ]:
df = load_data("https://raw.githubusercontent.com/AyushmanRaha/customer_churn_prediction/refs/heads/main/data/customer_data.csv")

Succesfully loaded data from: https://raw.githubusercontent.com/AyushmanRaha/customer_churn_prediction/refs/heads/main/data/customer_data.csv


**Calling the initial_exploration() function on the dataFrame.**

In [ ]:
initial_exploration(df)


🔍 DATAFRAME OVERVIEW
Shape: 7043 rows x 21 columns

📋 INFO
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-nu

**Creating the modified dataFrame**

In [ ]:
df = clean_and_correct_data(df, dropped_columns=['customerid'], numeric_columns=['totalcharges', 'monthlycharges', "tenure"])


Data Cleaning Summary:
--------------------------------------------------
Dropped Columns:
1) customerid
--------------------------------------------------
Null values remaining in the DataFrame: False
--------------------------------------------------
Shape: 7043 rows x 20 columns
--------------------------------------------------
Updated Column list:
1) gender, dtype: object
2) seniorcitizen, dtype: int64
3) partner, dtype: object
4) dependents, dtype: object
5) tenure, dtype: int64
6) phoneservice, dtype: object
7) multiplelines, dtype: object
8) internetservice, dtype: object
9) onlinesecurity, dtype: object
10) onlinebackup, dtype: object
11) deviceprotection, dtype: object
12) techsupport, dtype: object
13) streamingtv, dtype: object
14) streamingmovies, dtype: object
15) contract, dtype: object
16) paperlessbilling, dtype: object
17) paymentmethod, dtype: object
18) monthlycharges, dtype: float64
19) totalcharges, dtype: float64
20) churn, dtype: object
------------------------

In [ ]:
df.head() # Checking the first 5 values of the dataFrame after modification

,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,onlinebackup,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,monthlycharges,totalcharges,churn
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


**checkingValuesInColumns() - Used to check what values and how many are there under each column in the DataFrame.**

In [ ]:
def checkingValuesInColumns():
  for column in df.columns:
    print(f"Value Counts for '{column}':\n")

    print(df[column].value_counts(normalize=True)*100)
    print("-" * (len(column) + 24))
    print(" ")
    print(" ")

**Creating the new DataFrame with analyze_churn_permutations(df) function**

In [ ]:
new_df = analyze_churn_permutations(df)
new_df.shape

(2190, 9)

In [ ]:
new_df.head()

,Serial No.,gender,age category,partner,dependents,services opted,Churn %,Churned,Not Churned
0,1,Female,Non-senior,No,No,"contract, deviceprotection, internetservice, m...",0.00,0,3
1,2,Female,Non-senior,No,No,"contract, deviceprotection, internetservice, m...",50.00,1,1
2,3,Female,Non-senior,No,No,"contract, deviceprotection, internetservice, m...",28.57,2,5
3,4,Female,Non-senior,No,No,"contract, deviceprotection, internetservice, m...",5.56,1,17
4,5,Female,Non-senior,No,No,"contract, deviceprotection, internetservice, m...",0.00,0,2


**Creating the verification table to see whether the data in the new DataFrame matches with the original source DataFrame.**

In [ ]:
verification_table = verify_churn_logic(df)
print(verification_table)

Total permutations generated: 2190
   Permutation Index                           Profile                           Services  Code Churned  Actual Churned  Code Not Churned  Actual Not Churned  Match
0               1172      Male, Non-senior, P:No, D:No  contract, count, deviceprotect...             0               0                 2                   2   True
1               1405     Male, Non-senior, P:No, D:Yes  contract, count, internetservi...             2               2                 1                   1   True
2                452   Female, Non-senior, P:Yes, D:No  contract, count, deviceprotect...             1               1                 3                   3   True
3               1126      Male, Non-senior, P:No, D:No  contract, count, deviceprotect...             0               0                 1                   1   True
4                581  Female, Non-senior, P:Yes, D:Yes  contract, count, deviceprotect...             0               0                 1   

# **Exploratory Data Analysis**

EDA 1: Identify high risk "exit points" based on how customers are billed.

EDA 2: Determine the "Danger Zone" - the specific time frame and price point where customers are most likely to drop off.

EDA 3: Quantify how "bundled services" affect cutomer loyalty.